In [1]:
import re

In [2]:
import numpy as np
import pandas as pd
import emoji

C:\Users\dell\AppData\Local\Temp\ipykernel_11768\2645341026.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


ModuleNotFoundError: No module named 'emoji'

In [3]:
pip install emoji


   ---------------------------------------- 0.0/421.5 kB ? eta -:--:--
   ---------------------------------------- 0.0/421.5 kB ? eta -:--:--
   ---------------------------------------- 0.0/421.5 kB ? eta -:--:--
   ----- ---------------------------------- 61.4/421.5 kB 1.6 MB/s eta 0:00:01
   -------- ------------------------------- 92.2/421.5 kB 1.0 MB/s eta 0:00:01
   --------------------- ------------------ 225.3/421.5 kB 1.5 MB/s eta 0:00:01
   ---------------------------------- ----- 358.4/421.5 kB 1.9 MB/s eta 0:00:01
   ---------------------------------- ----- 358.4/421.5 kB 1.9 MB/s eta 0:00:01
   ---------------------------------- ----- 358.4/421.5 kB 1.9 MB/s eta 0:00:01
   ------------------------------------- -- 399.4/421.5 kB 1.2 MB/s eta 0:00:01
   ---------------------------------------  419.8/421.5 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------  419.8/421.5 kB 1.1 MB/s eta 0:00:01
   -------------------------------------- 421.5/421.5 kB 876.9 kB/s 

In [4]:
import emoji

In [23]:
from nltk.stem import PorterStemmer 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [6]:
data= pd.read_csv('Tweets.csv')

In [7]:
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [8]:
data.shape

(14640, 15)

In [11]:
confidence_threshold= 0.6
data= data.drop(data.query('airline_sentiment_confidence < @confidence_threshold').index, axis=0).reset_index(drop=True)

In [14]:
tweets_df= pd.concat([data['text'], data['airline_sentiment']], axis=1, )

In [15]:
tweets_df

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica I didn't today... Must mean I n...,neutral
2,@VirginAmerica it's really aggressive to blast...,negative
3,@VirginAmerica and it's a really big bad thing...,negative
4,@VirginAmerica seriously would pay $30 a fligh...,negative
...,...,...
14397,@AmericanAir right on cue with the delays👌,negative
14398,@AmericanAir leaving over 20 minutes Late Flig...,negative
14399,@AmericanAir Please bring American Airlines to...,neutral
14400,"@AmericanAir you have my money, you change my ...",negative


In [16]:
sentence_ordering=['negative','neutral', 'positive']
tweets_df['airline_sentiment']= tweets_df['airline_sentiment'].apply(lambda x: sentence_ordering.index(x) )

In [17]:
tweets_df

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,1
1,@VirginAmerica I didn't today... Must mean I n...,1
2,@VirginAmerica it's really aggressive to blast...,0
3,@VirginAmerica and it's a really big bad thing...,0
4,@VirginAmerica seriously would pay $30 a fligh...,0
...,...,...
14397,@AmericanAir right on cue with the delays👌,0
14398,@AmericanAir leaving over 20 minutes Late Flig...,0
14399,@AmericanAir Please bring American Airlines to...,1
14400,"@AmericanAir you have my money, you change my ...",0


In [24]:
ps= PorterStemmer()

def process_tweet(tweet):
    new_tweet= tweet.lower()
    new_tweet= re.sub(r'@\w+', '',new_tweet)
    new_tweet= re.sub(r'#', '',new_tweet)
    new_tweet= re.sub(r':', ' ',emoji.demojize(new_tweet))
    new_tweet = re.sub(r'http\S+', '',new_tweet) # Remove URLs
    new_tweet = re.sub(r'\$\S+', 'dollar', new_tweet) # Change dollar amounts to dollar
    new_tweet = re.sub(r'[^a-z0-9\s]', '', new_tweet) # Remove punctuation
    new_tweet = re.sub(r'[0-9]+', 'number', new_tweet) # Change number values to number
    new_tweet = new_tweet.split(" ")
    new_tweet = list(map(lambda x: ps.stem(x), new_tweet)) # Stemming the words
    new_tweet = list(map(lambda x: x.strip(), new_tweet)) # Stripping whitespace from the words
    if '' in new_tweet:
        new_tweet.remove('')
    return new_tweet
    

In [25]:
tweets = tweets_df['text'].apply(process_tweet)

labels = np.array(tweets_df['airline_sentiment'])

In [26]:
tweets

0                                           [what, , said]
1        [i, didnt, today, must, mean, i, need, to, tak...
2        [it, realli, aggress, to, blast, obnoxi, enter...
3         [and, it, a, realli, big, bad, thing, about, it]
4        [serious, would, pay, dollar, a, flight, for, ...
                               ...                        
14397           [right, on, cue, with, the, delay, hand, ]
14398    [leav, over, number, minut, late, flight, no, ...
14399    [pleas, bring, american, airlin, to, blackberr...
14400    [you, have, my, money, you, chang, my, flight,...
14401    [we, have, number, ppl, so, we, need, number, ...
Name: text, Length: 14402, dtype: object

In [27]:
vocabulary = set()

for tweet in tweets:
    for word in tweet:
        if word not in vocabulary:
            vocabulary.add(word)

vocab_length = len(vocabulary)


max_seq_length = 0

for tweet in tweets:
    if len(tweet) > max_seq_length:
        max_seq_length = len(tweet)


print("Vocab length:", vocab_length)
print("Max sequence length:", max_seq_length)

Vocab length: 11250
Max sequence length: 90


In [28]:
tokenizer = Tokenizer(num_words=vocab_length)
tokenizer.fit_on_texts(tweets)

sequences = tokenizer.texts_to_sequences(tweets)

word_index = tokenizer.word_index

model_inputs = pad_sequences(sequences, maxlen=max_seq_length, padding='post')

In [29]:
model_inputs

array([[  49,    2,  218, ...,    0,    0,    0],
       [   5,  191,  102, ...,    0,    0,    0],
       [  15,  138, 2841, ...,    0,    0,    0],
       ...,
       [  76,  507,  435, ...,    0,    0,    0],
       [   8,   19,   12, ...,    0,    0,    0],
       [  37,   19,    4, ...,    0,    0,    0]])

In [30]:
model_inputs.shape

(14402, 90)

In [31]:
X_train, X_test, y_train, y_test = train_test_split(model_inputs, labels, train_size=0.7, random_state=22)

In [36]:
embedding_dim = 32


inputs = tf.keras.Input(shape=(max_seq_length,))

embedding = tf.keras.layers.Embedding(
    input_dim=vocab_length,
    output_dim=embedding_dim,
    
)(inputs)


# Model A (just a Flatten layer)
flatten = tf.keras.layers.Flatten()(embedding)

# Model B (GRU with a Flatten layer)
gru = tf.keras.layers.GRU(units=embedding_dim)(embedding)
gru_flatten = tf.keras.layers.Flatten()(gru)

# Both A and B are fed into the output
concat = tf.keras.layers.concatenate([flatten, gru_flatten])

outputs = tf.keras.layers.Dense(3, activation='softmax')(concat)


model = tf.keras.Model(inputs, outputs)

tf.keras.utils.plot_model(model)

You must install pydot (`pip install pydot`) for `plot_model` to work.


In [37]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)



batch_size = 32
epochs = 100

history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True,
            verbose=1
        ),
        tf.keras.callbacks.ReduceLROnPlateau()
    ]
)

Epoch 1/100
252/252 ━━━━━━━━━━━━━━━━━━━━ 19s 42ms/step - accuracy: 0.6176 - loss: 0.8769 - val_accuracy: 0.7194 - val_loss: 0.6568 - learning_rate: 0.0010
Epoch 2/100
252/252 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.7826 - loss: 0.5501 - val_accuracy: 0.7799 - val_loss: 0.5317 - learning_rate: 0.0010
Epoch 3/100
252/252 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.8687 - loss: 0.3827 - val_accuracy: 0.7893 - val_loss: 0.5039 - learning_rate: 0.0010
Epoch 4/100
252/252 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9097 - loss: 0.2795 - val_accuracy: 0.8022 - val_loss: 0.5024 - learning_rate: 0.0010
Epoch 5/100
252/252 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9371 - loss: 0.2100 - val_accuracy: 0.7987 - val_loss: 0.5126 - learning_rate: 0.0010
Epoch 6/100
252/252 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9548 - loss: 0.1599 - val_accuracy: 0.7972 - val_loss: 0.5354 - learning_rate: 0.0010
Epoch 7/100
252/252 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9

In [38]:
model.evaluate(X_test, y_test)

136/136 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.8002 - loss: 0.5118


[0.4932244122028351, 0.8060634136199951]